In [1]:
import pandas as pd

In [105]:
df = pd.read_csv('ratings_Electronics.csv',nrows=200000,header=None)

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
0    200000 non-null object
1    200000 non-null object
2    200000 non-null float64
3    200000 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 4.6+ MB


In [107]:
df.columns = ['userid','productid','rating','timestamp']

In [108]:
df.head()

,userid,productid,rating,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [109]:
df.drop('timestamp',axis=1,inplace=True)

In [110]:
df.head()

,userid,productid,rating
0,AKM1MP6P0OYPR,0132793040,5.0
1,A2CX7LUOHB2NDG,0321732944,5.0
2,A2NWSAGRHCP8N5,0439886341,1.0
3,A2WNBOD3WNDNKT,0439886341,3.0
4,A1GI0U4ZRJA8WN,0439886341,1.0


In [168]:
df.isna().sum()

userid       0
productid    0
rating       0
freq         0
dtype: int64

In [111]:
df['freq'] = df.groupby('userid')['userid'].transform('count')

In [112]:
df.head()

,userid,productid,rating,freq
0,AKM1MP6P0OYPR,0132793040,5.0,1
1,A2CX7LUOHB2NDG,0321732944,5.0,1
2,A2NWSAGRHCP8N5,0439886341,1.0,1
3,A2WNBOD3WNDNKT,0439886341,3.0,1
4,A1GI0U4ZRJA8WN,0439886341,1.0,1


In [113]:
df_small = df[df['freq']>50]

In [114]:
df_small.head()

,userid,productid,rating,freq
2162,A5JLAU2ARJ0BO,1400532655,1.0,86
11036,A231WM2Z2JL0U3,B00000J1US,5.0,192
12858,AT2J7H5TRZM8Z,B00000J4FS,5.0,53
13525,A231WM2Z2JL0U3,B00000J570,4.0,192
16138,A231WM2Z2JL0U3,B00000JFJA,3.0,192


In [115]:
from surprise import KNNWithMeans
from surprise import Dataset, Reader
from surprise import accuracy
from surprise.model_selection import train_test_split

In [116]:
reader = Reader(rating_scale=(1, 5))

In [117]:
data = Dataset.load_from_df(df_small[['userid','productid','rating']], reader)

In [118]:
data

In [119]:
trainset, testset = train_test_split(data, test_size=.3)

In [120]:
user_records = trainset.ur
user_records

defaultdict(list,
            {0: [(0, 5.0),
              (2, 5.0),
              (3, 5.0),
              (10, 5.0),
              (18, 5.0),
              (21, 5.0),
              (26, 5.0),
              (30, 5.0),
              (32, 5.0),
              (33, 5.0),
              (38, 5.0),
              (42, 5.0),
              (45, 5.0),
              (61, 5.0),
              (74, 5.0),
              (76, 5.0),
              (82, 5.0),
              (85, 5.0),
              (92, 5.0),
              (96, 5.0),
              (99, 5.0),
              (104, 5.0),
              (108, 5.0),
              (110, 5.0),
              (113, 5.0),
              (120, 5.0),
              (123, 5.0),
              (129, 5.0),
              (133, 5.0),
              (139, 5.0),
              (142, 5.0),
              (143, 5.0),
              (147, 5.0),
              (153, 5.0),
              (175, 5.0),
              (183, 5.0),
              (186, 5.0),
              (193, 5.0),
              (

In [121]:
# Let's locate what is the user id 220 is actually in main dataset 
trainset.to_raw_uid(0)

'A25HBO5V8S8SEA'

In [122]:
trainset.to_raw_iid(1)

'B00004TVSO'

In [123]:
df[df.productid == "B00000JFJA"].head()

,userid,productid,rating,freq
16133,A2H1S4BHHZB5S9,B00000JFJA,4.0,1
16134,A37AJKN9EYGVQ4,B00000JFJA,5.0,1
16135,A188C7ZSTU62ZR,B00000JFJA,5.0,1
16136,A3ET8UZUWY0KV3,B00000JFJA,1.0,2
16137,A6E6JUISCNYGG,B00000JFJA,5.0,1


# As the suprise dataset appears working properly, let's now build the Collaborative Filtering model

In [124]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [125]:
# run the trained model against the testset
test_pred = algo.test(testset)

In [126]:
test_pred

[Prediction(uid='A231WM2Z2JL0U3', iid='B00004SCKA', r_ui=4.0, est=4.4043887147335425, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid='A5JLAU2ARJ0BO', iid='B00006HXF4', r_ui=2.0, est=4.1875, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid='A231WM2Z2JL0U3', iid='B00005RG4N', r_ui=3.0, est=4.4043887147335425, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid='A231WM2Z2JL0U3', iid='B00006B1HC', r_ui=5.0, est=4.4043887147335425, details={'was_impossible': True, 'reason': 'User and/or item is unkown.'}),
 Prediction(uid='A5JLAU2ARJ0BO', iid='B00005QFZG', r_ui=5.0, est=4.1875, details={'actual_k': 0, 'was_impossible': False}),
 Prediction(uid='A2BGZ52M908MJY', iid='B00005QJCU', r_ui=3.0, est=3.7288461538461535, details={'actual_k': 1, 'was_impossible': False}),
 Prediction(uid='A231WM2Z2JL0U3', iid='B00005MEMN', r_ui=4.0, est=4.4043887147335425, details={'was_impossible': True, 'reason': 

In [127]:
# get RMSE
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Item-based Model : Test Set
RMSE: 1.0643


1.0643384806134557

# Popularity based filtering model

In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split

In [129]:
vectorizer = TfidfVectorizer()

In [132]:
df_small1 = df_small.copy()

In [133]:
# Because I was getting error for too large array while aplying liner 

In [134]:
df_small1.groupby('productid')['rating'].mean().sort_values(ascending=False).head()  


productid
B00006JN3G    5.0
B000062TTE    5.0
B00005KAMT    5.0
B00005KB24    5.0
B00005LAEP    5.0
Name: rating, dtype: float64

In [135]:
df_small1.groupby('productid')['rating'].count().sort_values(ascending=False).head()  

productid
B00005QFZG    3
B00006412G    2
B00003G1RG    2
B00005B70I    2
B00004SB92    2
Name: rating, dtype: int64

In [136]:
ratings_mean_count = pd.DataFrame(df_small1.groupby('productid')['rating'].mean()) 

In [137]:
ratings_mean_count

,rating
productid,
1400532655,1.0
B00000J1US,5.0
B00000J4FS,5.0
B00000J570,4.0
B00000JFJA,3.0
B00000JSGF,4.0
B00000K4KH,5.0
B00001OPK8,4.0
B00001XE0L,5.0


In [138]:
ratings_mean_count['rating_counts'] = pd.DataFrame(df_small1.groupby('productid')['rating'].count())  

In [139]:
ratings_mean_count.head(10)

,rating,rating_counts
productid,,
1400532655,1.0,1
B00000J1US,5.0,1
B00000J4FS,5.0,1
B00000J570,4.0,1
B00000JFJA,3.0,1
B00000JSGF,4.0,1
B00000K4KH,5.0,1
B00001OPK8,4.0,1
B00001XE0L,5.0,1


In [140]:
ratings_mean_count.sort_values(by='rating_counts', ascending=False)

,rating,rating_counts
productid,,
B00005QFZG,4.666667,3
B00004SCKA,4.500000,2
B00005ICE1,5.000000,2
B00006HXF4,3.500000,2
B0000645C9,4.500000,2
B0000645C8,4.500000,2
B00004S9AK,5.000000,2
B00005AW1X,3.500000,2
B00004SB92,4.500000,2


In [174]:
from sklearn.feature_extraction.text import CountVectorizer
df_small2 = df_small.copy()

In [175]:
count = CountVectorizer()
count_matrix = count.fit_transform(df_small2['productid'])

In [176]:
count_matrix

<457x411 sparse matrix of type '<class 'numpy.int64'>'
	with 457 stored elements in Compressed Sparse Row format>

In [177]:
count_vect_df = pd.DataFrame(count_matrix.todense(), columns=count.get_feature_names())

count_vect_df

,1400532655,b00000j1us,b00000j4fs,b00000j570,b00000jfja,b00000jsgf,b00000k4kh,b00001opk8,b00001xe0l,b00001yvg4,...,b00006ir39,b00006is65,b00006jhtq,b00006ji6t,b00006ji6u,b00006jkxn,b00006jlwj,b00006jm1w,b00006jn2r,b00006jn3g
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [179]:
indices = pd.Series(df_small2.index)
indices[:5]

0     2162
1    11036
2    12858
3    13525
4    16138
dtype: int64

In [183]:
type(indices)

pandas.core.series.Series

In [181]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [184]:
# function that takes in movie title as input and returns the top 20 recommended products
def recommendations(product, cosine_sim = cosine_sim):
    
    recommended_prod = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == product].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 5 most similar products
    top_5_indexes = list(score_series.iloc[1:6].index)
    print(top_5_indexes)
    
    # populating the list with top 5 matching products
    for i in top_5_indexes:
        recommended_prod.append(list(df.index)[i])
        
    return recommended_prod

In [185]:
indices[indices == '1400532655'].index[0]

C:\Users\ADMIN\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


IndexError: index 0 is out of bounds for axis 0 with size 0

In [186]:
df_small2.head(10)

,userid,productid,rating,freq
2162,A5JLAU2ARJ0BO,1400532655,1.0,86
11036,A231WM2Z2JL0U3,B00000J1US,5.0,192
12858,AT2J7H5TRZM8Z,B00000J4FS,5.0,53
13525,A231WM2Z2JL0U3,B00000J570,4.0,192
16138,A231WM2Z2JL0U3,B00000JFJA,3.0,192
17720,A231WM2Z2JL0U3,B00000JSGF,4.0,192
19752,A231WM2Z2JL0U3,B00000K4KH,5.0,192
20435,A231WM2Z2JL0U3,B00001OPK8,4.0,192
28531,A2BGZ52M908MJY,B00001XE0L,5.0,54
28594,A231WM2Z2JL0U3,B00001YVG4,5.0,192


In [187]:
recommendations('B00000K4KH')

IndexError: index 0 is out of bounds for axis 0 with size 0